## Lab | Making predictions with logistic regression
#### In this lab, you will be using the Sakila database of movie rentals.

#### In order to optimize our inventory, we would like to know which films will be rented. We are asked to create a model to predict it. So we use the information we have from May 2005 to create the model.

#### Instructions

#### Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).

#### Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable.

#### Read the data into a Pandas dataframe. At this point you should have 1000 rows. Number of columns depends on the number of features you chose.

#### Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.

#### Create a logistic regression model to predict 'rented_in_may' from the cleaned data.
#### Evaluate the results.

In [ ]:
# prep: import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [ ]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank' # localhost= 127.000.001
engine = create_engine(connection_string)
query = '''SELECT DISTINCT f. title,
SUM(
	CASE
		WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN 1
        else 0
	END) AS rented_in_may
FROM sakila.film f
LEFT JOIN sakila.inventory i
USING (film_id) 
LEFT JOIN sakila.rental r 
ON i.inventory_id = r.inventory_id
GROUP BY f.film_id;'''

data = pd.read_sql_query(query, engine)
data.head(5)

In [ ]:
def boolean(x):
    if x>=1:
        return 1
    else:
        return 0

data['rented_in_may'] = list(map(boolean, data['rented_in_may']))

In [ ]:
data.head(5)

In [ ]:
# Need to X-y-split AND train-test-split BEFORE I apply transformations, 
# then train transformation on training set only
y = data['rented_in_may']
X = data.drop('rented_in_may', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [ ]:
import numpy as np
from pyts.preprocessing import MaxAbsScaler
# from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(include = np.number)

# Scaling data
transformer = MaxAbsScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)

In [ ]:
from sklearn.linear_model import LogisticRegression  # LogisticRegression...neu
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train_transformed, y_train) # 'auto','ova' or 'multinomial'